In [1]:
#r "nuget:XPlot.Plotly,3.0.1"
#r "nuget:MathNet.Numerics,4.12.0"
#r "nuget:System.Drawing.Common,5.0.0"
using System;
using MathNet.Numerics.LinearAlgebra;
using System.Linq;
using System.IO;
using System.Drawing;
using System.Drawing.Imaging;
using XPlot.Plotly;

In [1]:
public static void Activate(this Matrix<double> m)
{
    m.Map(elem => ActivationFunc(elem), m);
}

public static double ActivationFunc(double x)
{
    if (x > 0) return 1;
    if (x < 0) return -1;
    return x;
}

In [1]:
public static void Randomize(this Matrix<double> m)
{
    var rand = new Random();

    for (int i = 0; i < m.RowCount; i++)
    {
        for (int j = 0; j < m.ColumnCount; j++)
        {
            m[i, j] = Math.Clamp(m.At(i, j) + rand.Next(-1, 1), -1, 1);
        }
    }
}
public static void Randomize(this Matrix<double>[] m)
{
    var rand = new Random();
    foreach (var matrix in m)
    {
        for (int i = 0; i < matrix.RowCount; i++)
        {
            for (int j = 0; j < matrix.ColumnCount; j++)
            {
                var needAbs = rand.NextDouble();
                var noise = needAbs >= 0.9 ? Math.Abs(matrix.At(i, j)) : matrix.At(i, j);
                matrix[i, j] = noise;
            }
        }
    }
}

In [1]:
public static Bitmap ToImage(this Matrix<double> m, string filePath)
{
    var size = (int)Math.Sqrt(m.ColumnCount * m.RowCount);
    Bitmap image = new Bitmap(size, size);
    m = Matrix<double>.Build.DenseOfRowMajor(size, size, m.ToRowMajorArray());

    for (int i = 0; i < size; i++)
    {
        for (int j = 0; j < size; j++)
        {
            var val = 255 - (int)m.At(i, j) * 255;
            val = Math.Clamp(val, 0, 255);
            Color col = Color.FromArgb(val, val, val);
            image.SetPixel(j, i, col);
        }
    }
    image.Save($"{filePath}.bmp", ImageFormat.Bmp);
    return image;
}

public static void ToImage(this Matrix<double>[] m, string filePath)
{
    var size = (int)Math.Sqrt(m[0].ColumnCount * m[0].RowCount);
    var index = 0;

    foreach (var matrix in m)
    {
        Bitmap image = new Bitmap(size, size);
        var matrix1 = Matrix<double>.Build.DenseOfRowMajor(size, size, matrix.ToRowMajorArray());
        for (int i = 0; i < size; i++)
        {
            for (int j = 0; j < size; j++)
            {
                var val = 255 - (int)matrix1.At(i, j) * 255;
                val = Math.Clamp(val, 0, 255);
                Color col = Color.FromArgb(val, val, val);
                image.SetPixel(j, i, col);
            }
        }
        image.Save($"{filePath}_{index++}.bmp", ImageFormat.Bmp);
    }
}

In [1]:
public static Matrix<double> Join(this Matrix<double>[] matrices)
{
    var m = matrices[0];
    var vecBuilder = Vector<double>.Build;
    for (int i = 1; i < matrices.Length; i++)
    {
        m = m.InsertColumn(m.ColumnCount, vecBuilder.DenseOfArray(matrices[i].ToColumnMajorArray()));
    }
    return m;
}

In [1]:
public static Matrix<double>[] ReadInput(params string[] inputFiles)
{
    var matrixBuilder = Matrix<double>.Build;
    List<Matrix<double>> matrices = new();

    foreach (var filePath in inputFiles)
    {
        var rawInput = File.ReadAllLines(filePath);
        var inputAsNum = rawInput.Select(row => row.Split(" ").Select(stringNum => double.Parse(stringNum)).ToArray()).ToArray();
        var matrix = matrixBuilder.DenseOfRowArrays(inputAsNum);
        matrix.ToImage($"{filePath}");
        matrix = matrixBuilder.Dense(matrix.ColumnCount * matrix.RowCount, 1, matrix.ToRowMajorArray());
        matrices.Add(matrix);
    }

    return matrices.ToArray();
}

In [1]:
public static bool Compare(this Matrix<double> matrix, Matrix<double> other)
{
    for (int i = 0; i < matrix.RowCount; i++)
    {
        for (int j = 0; j < matrix.ColumnCount; j++)
        {
            if (matrix[i, j] != other[i, j])
            {
                return false;
            }
        }
    }
    return true;
}

In [1]:
class Network
    {
        private Matrix<double> Weights { get; set; }

        public Network(Matrix<double> input)
        {
            var matrixBuilder = Matrix<double>.Build;
            Weights = matrixBuilder.Dense(input.RowCount, input.RowCount, 0);

            Train(input);
        }

        public void Train(Matrix<double> input)
        {

            //Hebbian(input);
            //Projections(input);
            DeltaProjections(input);

            for (int i = 0; i < Weights.RowCount; i++)
            {
                Weights[i, i] = 0;
            }

            Console.WriteLine($"Weight matrix is:\n{Weights}");
        }

        private void Hebbian(Matrix<double> input)
        {
            for(int i = 0; i < input.ColumnCount; i++)
            {
                var vec = input.Column(i).ToColumnMatrix();
                var deltaMatrix = vec * vec.Transpose();
                Weights += deltaMatrix;
            }
        }

        private void Projections(Matrix<double> input)
        {
            Matrix<double> oldWeights;

            // projections method
            oldWeights = Weights.Map(f => f);
            var t1 = oldWeights * input - input;
            var t2 = t1.Transpose();
            var coeff = 1f / (input.Transpose() * input - input.Transpose() * oldWeights * input)[0, 0];
            Weights += coeff * t1 * t2;
        }

        private void DeltaProjections(Matrix<double> input)
        {
            double e = 1E-8;
            double change = 0;
            double h = 0.8;
            double delta = 0;
            double previousDelta = 0;
            
            do
            {
                delta = 0;
                for(int i = 0; i < input.ColumnCount; i++)
                {
                    var vec = input.Column(i).ToColumnMatrix();
                    var deltaW = (vec - Weights * vec) * vec.Transpose() * (h / input.RowCount.Sqrt());
                    Weights += deltaW;
                    delta = deltaW.RowAbsoluteSums().Sum();
                }
                change = Math.Abs(previousDelta - delta);
                previousDelta = delta;
                Console.WriteLine($"Change is: {change}");
            } while(change > e);
        }

        public Bitmap Process(Matrix<double> input)
        {
            bool isDenoised = false;
            while (!isDenoised)
            {
                isDenoised = true;
                var row = input.Transpose();
                var y = row * Weights;
                y.Activate();
                input.SetColumn(0, y.ToColumnMajorArray());
                isDenoised &= row.Compare(y);
                Console.WriteLine($"Energy is: {GetError(y.Transpose())}");
            }

            return input.ToImage("denoised");
        }

        public Matrix<double> GetError(Matrix<double> input)
        {
            return - 1.0 / 2 * input.Transpose() * Weights * input;
        }
    }

In [1]:
public static double Abs(this double value)
{
    return Math.Abs(value);
}

public static double Sqrt(this int value)
{
    return Math.Sqrt(value);
}

In [1]:
var input = ReadInput("1.txt", "2.txt", "3.txt", "4.txt", "5.txt");
Network network = new(input.Join());
input.Randomize();
input.ToImage("noise");


Weight matrix is:
DenseMatrix 400x400-Double
     0  0.0125  0.0075  0.0075  0.0075  0.0125  0.0125  ..  0.0125  0.0125
0.0125       0  0.0075  0.0075  0.0075  0.0125  0.0125  ..  0.0125  0.0125
0.0075  0.0075       0  0.0125  0.0125  0.0075  0.0075  ..  0.0075  0.0075
0.0075  0.0075  0.0125       0  0.0125  0.0075  0.0075  ..  0.0075  0.0075
0.0075  0.0075  0.0125  0.0125       0  0.0075  0.0075  ..  0.0075  0.0075
0.0125  0.0125  0.0075  0.0075  0.0075       0  0.0125  ..  0.0125  0.0125
0.0125  0.0125  0.0075  0.0075  0.0075  0.0125       0  ..  0.0125  0.0125
0.0125  0.0125  0.0075  0.0075  0.0075  0.0125  0.0125  ..  0.0125  0.0125
    ..      ..      ..      ..      ..      ..      ..  ..      ..      ..
0.0125  0.0125  0.0075  0.0075  0.0075  0.0125  0.0125  ..  0.0125  0.0125
0.0125  0.0125  0.0075  0.0075  0.0075  0.0125  0.0125  ..  0.0125  0.0125
0.0125  0.0125  0.0075  0.0075  0.0075  0.0125  0.0125  ..       0  0.0125
0.0125  0.0125  0.0075  0.0075  0.0075  0.0125  0.0125 

In [1]:
network.Process(input[3])

Energy is: DenseMatrix 1x1-Double
-240.85



Energy is: DenseMatrix 1x1-Double
-240.85



Tag,PhysicalDimension,Size,Width,Height,HorizontalResolution,VerticalResolution,Flags,RawFormat,PixelFormat,FrameDimensionsList,Palette,PropertyIdList,PropertyItems
<null>,"{Width=20, Height=20}","{Width=20, Height=20}",20,20,0,0,2,MemoryBMP,Format32bppArgb,[ 7462dc86-6180-4c7e-8e3f-ee7333a7a483 ],System.Drawing.Imaging.ColorPalette,[ ],[ ]
